# Library

In [72]:
import pandas as pd
import numpy as np
from tabulate import tabulate
import matplotlib.pyplot as plt
import time
from datetime import datetime, timedelta
import quandl
from datetime import datetime
from openpyxl import load_workbook

# Data Set

In [73]:
# GitHub raw file URL
xlsx_url = "https://raw.githubusercontent.com/elcacique69/DCF---Portfolio-Acquisition-Tool/main/Data_Set_Closing.xlsx"

df = pd.read_excel(xlsx_url, sheet_name="Planned Portfolio")

# Proportion of leased equipment

In [74]:
rows = len(df)

# use boolean indexing to select rows where 'Current status' is 'On Lease'.
# df["Current status"] == "On lease" returns a Boolean Series where each item is True if the corresponding 
# 'Current status' is 'On lease', and False otherwise.
# df[boolean_series] then returns a DataFrame containing only the rows where the Boolean Series is True.
# finally, len(df[boolean_series]) counts the number of such rows.

leased = len(df[df["Current Status"] == "On lease"])

# calculate the proportion of leased rows to the total rows
# this is done by dividing the number of 'leased' rows by the total number of rows

equipment_leased = leased / rows
equipment_not_leased = 1 - equipment_leased


# if the proportion is equal to 1, print "The portfolio is completely leased."

if equipment_leased == 1:
    print("The portfolio is completely leased.")
else:
    print("The portfolio is not completely leased.")
    
# Calculate and add the sum of Purchase Price for each container type
container_types = ["20'DC", "40'DC", "40'HC"]
container_sum_data = []
for container_type in container_types:
    filtered_df = non_leased_df[non_leased_df['Type'] == container_type]
    sum_purchase_price = filtered_df['Purchase Price'].sum()
    container_sum_data.append({'Metric': f'{container_type} Purchase Price', 'Value': sum_purchase_price})

container_sum_df = pd.DataFrame(container_sum_data)

# print the proportion of 'On lease' rows in the portfolio, multiplied by 100 to convert the proportion to a percentage
print(f"The proportion of 'On lease' in the portfolio is {equipment_leased * 100}%.")
print(f"The proportion of 'Off lease' in the portfolio is {equipment_not_leased * 100}%.")

The portfolio is not completely leased.
The proportion of 'On lease' in the portfolio is 94.01964972234089%.
The proportion of 'Off lease' in the portfolio is 5.980350277659118%.


# Data Set of non leased equipment

In [75]:
# First, create a DataFrame with only non-leased equipment
non_leased_df = df[df["Current Status"] != "On lease"]

# Get the list of container numbers for non-leased equipment
non_leased_container_numbers = non_leased_df["Serial Number"].tolist()

# Calculate the total NBV of non-leased equipment
total_non_leased_nbv = non_leased_df["Purchase Price"].sum()

# Calculate the total NBV of all equipment
total_nbv = df["Purchase Price"].sum()

# Calculate the proportion of NBV of non-leased equipment to the total NBV
non_leased_nbv_proportion = total_non_leased_nbv / total_nbv

# Create a list of results
results = [
    ["Total NBV of non-leased equipment", f"{total_non_leased_nbv:,.2f} USD"],
    ["NBV proportion of non-leased equipment", f"{non_leased_nbv_proportion * 100}%"],
]

# Output the results in a table format using tabulate
print(tabulate(results, headers=["Metric", "Value"], tablefmt="fancy_grid"))

╒════════════════════════════════════════╤════════════════════╕
│ Metric                                 │ Value              │
╞════════════════════════════════════════╪════════════════════╡
│ Total NBV of non-leased equipment      │ 613,480.00 USD     │
├────────────────────────────────────────┼────────────────────┤
│ NBV proportion of non-leased equipment │ 5.629027969773624% │
╘════════════════════════════════════════╧════════════════════╛


# Export the Off lease list to excel

In [77]:
# Specify the export file path for the new Excel file
export_path_non_leased = "/Users/carlosjosegonzalezacevedo/Documents/NEOMA/Thesis/DCF Container portfolio acquisition model/off_Lease_List.xlsx"

# Create a sample DataFrame for the Dashboard sheet
dashboard_data = {
    'Metric': ['Total NBV of non-leased equipment', 'NBV proportion of non-leased equipment'],
    'Value': [total_non_leased_nbv, non_leased_nbv_proportion]
}
dashboard_df = pd.DataFrame(dashboard_data)

# Create a new Excel file
writer = pd.ExcelWriter(export_path_non_leased, engine='xlsxwriter')

# Write the DataFrames to the respective sheets
non_leased_df.to_excel(writer, sheet_name='Non Leased Equipment', index=False)
dashboard_df.to_excel(writer, sheet_name='Dashboard', index=False)

container_sum_df.to_excel(writer, sheet_name='Dashboard', startrow=dashboard_df.shape[0]+2, index=False)

writer.save()